In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import imagesize

In [3]:
# %pip install imagesize

In [4]:
df = pd.read_csv('test.txt', header=None)

In [5]:
df.tail()

,0
8477,S24538 sub-S24538_ses-E50624_run-1_bp-chest_vp...
8478,S24539 sub-S24539_ses-E50625_run-1_bp-chest_vp...
8479,S24540 sub-S24540_ses-E50626_run-1_bp-chest_vp...
8480,S24540 sub-S24540_ses-E60362_run-1_bp-chest_vp...
8481,S24541 sub-S24541_ses-E50627_run-1_bp-chest_vp...


In [6]:
df = df[0].str.split(' ', expand=True)

In [7]:
df.rename(columns={0: "test_id", 1: 'filename', 2: 'test_result', 3: 'data_source'}, inplace=True)

In [8]:
df.head()

,test_id,filename,test_result,data_source
0,419639-003251,MIDRC-RICORD-1C-419639-003251-46647-0.png,positive,ricord
1,419639-001464,MIDRC-RICORD-1C-419639-001464-39871-0.png,positive,ricord
2,419639-000918,MIDRC-RICORD-1C-419639-000918-78965-0.png,positive,ricord
3,419639-003318,MIDRC-RICORD-1C-419639-003318-64285-0.png,positive,ricord
4,419639-001015,MIDRC-RICORD-1C-419639-001015-81591-0.png,positive,ricord


In [9]:
df.describe()

,test_id,filename,test_result,data_source
count,8482,8482,8482,8482
unique,4722,8482,2,3
top,S09382,MIDRC-RICORD-1C-419639-003251-46647-0.png,positive,bimcv
freq,38,1,4241,8082


In [10]:
df['full_filepath'] = df.apply(lambda row: os.path.join('test', row.filename), axis=1)

In [11]:
df.head()

,test_id,filename,test_result,data_source,full_filepath
0,419639-003251,MIDRC-RICORD-1C-419639-003251-46647-0.png,positive,ricord,test/MIDRC-RICORD-1C-419639-003251-46647-0.png
1,419639-001464,MIDRC-RICORD-1C-419639-001464-39871-0.png,positive,ricord,test/MIDRC-RICORD-1C-419639-001464-39871-0.png
2,419639-000918,MIDRC-RICORD-1C-419639-000918-78965-0.png,positive,ricord,test/MIDRC-RICORD-1C-419639-000918-78965-0.png
3,419639-003318,MIDRC-RICORD-1C-419639-003318-64285-0.png,positive,ricord,test/MIDRC-RICORD-1C-419639-003318-64285-0.png
4,419639-001015,MIDRC-RICORD-1C-419639-001015-81591-0.png,positive,ricord,test/MIDRC-RICORD-1C-419639-001015-81591-0.png


In [12]:
imagesize.get(df['full_filepath'][0])

(1024, 826)

In [ ]:
# width, height = imagesize.get("test.png")

In [13]:
df['x_shape'] = df.apply(lambda row: imagesize.get(row.full_filepath)[0], axis=1)
df['y_shape'] = df.apply(lambda row: imagesize.get(row.full_filepath)[1], axis=1)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
plt.hist(df['x_shape'])

In [ ]:
plt.hist(df['y_shape'])

In [ ]:
nan_count = df.isna().sum()
print(nan_count)

### TorchXRayVision Testing

In [ ]:
# !pip install torchxrayvision

In [ ]:
import torchxrayvision as xrv
import skimage, torch, torchvision


In [ ]:
import numpy as np
import skimage
import torch
import torchvision
import matplotlib.pyplot as plt
import torchxrayvision as xrv

In [ ]:
model = xrv.baseline_models.chestx_det.PSPNet()


In [ ]:
model

In [ ]:
# Read the image
img_path = "test/MIDRC-RICORD-1C-419639-003251-46647-0.png"  # change this to your image path
img = skimage.io.imread(img_path)

# Normalize the image
img = xrv.datasets.normalize(img, 255)

# Check if the image is already 2D (grayscale), then add a new axis
if img.ndim == 2:
    img = img[None, ...]  # Add a new axis to make it 3D
elif img.ndim == 3:
    # Convert RGB to grayscale if it's not already grayscale
    img = img.mean(2)[None, ...]  # Calculate the mean across the color channels

transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(512)])

img = transform(img)
img = torch.from_numpy(img)

In [ ]:
with torch.no_grad():
    pred = model(img)

In [ ]:
plt.figure(figsize = (26,5))
plt.subplot(1, len(model.targets) + 1, 1)
plt.imshow(img[0], cmap='gray')
for i in range(len(model.targets)):
    plt.subplot(1, len(model.targets) + 1, i+2)
    plt.imshow(pred[0, i])
    plt.title(model.targets[i])
    plt.axis('off')
plt.tight_layout()

In [ ]:
pred = 1 / (1 + np.exp(-pred))  # sigmoid
pred[pred < 0.5] = 0
pred[pred > 0.5] = 1

In [ ]:
plt.figure(figsize = (26,5))
plt.subplot(1, len(model.targets) + 1, 1)
plt.imshow(img[0], cmap='gray')
for i in range(len(model.targets)):
    plt.subplot(1, len(model.targets) + 1, i+2)
    plt.imshow(pred[0, i])
    plt.title(model.targets[i])
    plt.axis('off')
plt.tight_layout()

In [ ]:
x_values = []
y_values = []

for i in range(len(right_lung_pred)):
    for j in range(len(right_lung_pred[0])):
        if right_lung_pred[i][j] == 1:
            y_values.append([j])
            x_values.append([i])